In [ ]:
from typing import Annotated, TypedDict
from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    HumanMessage,
    SystemMessage,
)

from langchain_openai import ChatOpenAI
from langgraph.graph import END, START, StateGraph
from langgraph.graph.message import add_messages

In [ ]:
model = ChatOpenAI(model = 'gpt-5-2025-08-07')

In [ ]:
class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]

In [ ]:
generate_prompt = SystemMessage(
    "당신은 훌륭한 자기소개서를 작성하는 임무를 가진 자기소개서 작성 어시스턴트입니다."
    "사용자의 요청에 맞춰 최상의 자기소개서를 작성하세요"
    "사용자가 주제 및 맥락을 제공하면, 이전 시도에 대한 수정 버전을 응답하세요."
)

In [ ]:
reflection_prompt = SystemMessage(
    "당신은 자기소개서를 읽어보는 인사 담당자입니다., 사용자의 자기소개서를 읽고 비평이나 수정사항을 생성하세요"
    "변경해야하는 내용, 스타일, 깊이등 같이 구체적인 요구사항을 포함한 자세한 내용을 제공하세요 "
)

In [6]:
builder = StateGraph(State)

In [7]:
def generate(state: State) -> State:
    answer = model.invoke([generate_prompt] + state['messages'])
    return {'messages' : [answer]}

In [ ]:
def reflect(state : State) -> State:
    cls_map = {AIMessage :  HumanMessage, HumanMessage : AIMessage}
    translated = [reflection_prompt, state['messages'][0]] +[cls_map[msg.__class__](content=msg.content)   for msg in state['messages'][1:]]
    answer = model.invoke(translated)
    return {'messages' : [HumanMessage(content=answer.content)]}

In [ ]:
def should_continue(state : State) :
    if len(state['messages']) > 6 :
        return END
    else:
        return 'reflect'

In [10]:
builder.add_node('generate', generate)
builder.add_node('reflect', reflect)
builder.add_edge(START, 'generate')
builder.add_conditional_edges('generate', should_continue)
builder.add_edge('reflect', 'generate')
graph = builder.compile()

In [ ]:
graph.get_graph().draw_mermaid_png(output_file_path="./자소서.png")

b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\xec\x00\x00\x01\x08\x08\x02\x00\x00\x00\xd3\x13\x8fc\x00\x00\x00\x01sRGB\x00\xae\xce\x1c\xe9\x00\x00\x18WIDATx\x9c\xed\xddw\\\x13\xf7\xff\xc0\xf1\xcf\xe52%\t+\t\x08\x01\x14gU\x14,*u#Z7\x82\xfa\xc5\x85h\x1du\xa0_m\xc5\xf5m\x1d\xa5Z\\\xb8g\xebjk)\xb6\x15\xab\xd6j\xdd\xbb\xa0\xa8(\xb8q\x01\x01\xd9\x04\x12\x12\x92\x0b\xfc\xfe\x88?\x14\x0c\xc1\x87\x86\xdc}\xe4\xfd\xfc\x8b\xde\xe5\x927\xc9\xab\xe7%$\x17\xa2\xa2\xa2\x02\x01\x803\x16\xdd\x03\x00\xf0\xbe b\x80=\x88\x18`\x0f"\x06\xd8\x83\x88\x01\xf6 b\x80=v\xad\x97P<\xd6\xe4g\xe94%\x06\xab\xcc\xc3\\\\\x01Kh\xcb\x96\xc9y\xb6R\x0e\xdd\xb3\xd4N]L\xbdx\xa6-)\xa4\xcaJ\xcb\xe9\x9e\xe5\xdd\tD\xa4\xa4!\xd7\xa5\x89\xc0\xfc\xc5\x083\xaf\x13\xeb\xb4\xe5\x7fn\xcf\xe4pY\xb6\x12.\x97_\xdf\xf7\xd9<\x01\x99\x9d\xa6!\x10rr\xe7\xf9\xf6\xb1\xa7{\x1cs\x92/+\x9f\xa6\x94\x12$rr\x17\xe8\xb4\x18G\\\xa6)/\xc9\xd7QT\xf9\x90).\x1c^\x8d\x05\xd6\x18\xb1N[~\xe4\xfb,\x9f\x00G\xa9\x9c_\x97s\xe2\xe7\xca\xe1\x1cg\x0f^\xbb\xee\xb6

In [ ]:
initial_state = {
    'messages' : [
        HumanMessage(
            content="엔코아 고등학교, 엔코어 대학교, 봉사활동 100시간, 토익 900점, 경제 동아리 활동"
        )
    ]
}

In [ ]:
for ouput in graph.stream(initial_state):
    message_type = 'generate' if 'generate' in ouput else 'reflect'
    print("\nNew Message: ", ouput[message_type]['messages'][-1].content, ".....")


New Message:  문제를 숫자와 사람의 언어로 풀어내는 균형 잡힌 인재입니다. 엔코아 고등학교에서 기초 학업 역량을 다진 뒤 엔코어 대학교에서 학문적 토대를 확장해 왔고, 특히 경제 동아리 활동을 통해 시사 이슈를 꾸준히 분석·토론하며 논리적 사고와 자료 해석, 글쓰기 역량을 강화했습니다. 팀 프로젝트를 운영하며 의견을 구조화해 설득력 있게 전달하는 법을 배웠고, 서로의 관점을 조율하는 협업의 기본기를 체득했습니다.

토익 900점은 영문 자료를 신속하고 정확하게 파악하는 능력의 근거입니다. 해외 자료 리서치나 다국어 환경에서의 문서 커뮤니케이션 등 업무 상황에서 필요한 기초 체력을 갖추고 있어, 정보의 비대칭을 줄이고 의사결정의 속도를 높이는 데 기여할 수 있습니다.

봉사활동 100시간은 저의 성실함과 책임감을 보여 줍니다. 정기적으로 시간을 내어 참여하며 다양한 연령과 배경의 사람들을 만났고, 그 과정에서 공감 능력과 문제를 끝까지 마무리하는 지속성을 길렀습니다. 이는 조직 내외부 이해관계자와 신뢰를 쌓고, 작은 약속도 지키는 태도로 이어졌습니다.

저의 강점은 데이터와 텍스트를 빠르게 이해해 핵심을 도출하는 분석력, 상대의 관점을 고려해 메시지를 명확히 전달하는 커뮤니케이션, 그리고 합의된 목표를 실천으로 옮기는 실행력입니다. 경제 동아리에서 다진 사고력과 봉사로 다져진 책임감, 영어 기반의 정보 접근성을 바탕으로, 현장에서 근거 중심의 의사결정을 돕고 팀의 성과에 실질적으로 기여하겠습니다.

지원 분야나 회사/직무 정보를 알려주시면, 요구 역량에 맞춰 더 촘촘하게 다듬은 맞춤형 자기소개서로 수정해 드리겠습니다. .....

New Message:  총평
- 강점: 서두의 메시지(숫자와 사람의 언어)와 전반적 논리 흐름이 명확합니다. 문장력이 안정적이며, 협업·커뮤니케이션 역량을 균형 있게 어필했습니다.
- 보완 필요: 사례의 구체성이 부족해 차별화가 약합니다. 토익/봉사 강조가 장점 대비 비중이 크고, 직무 연계가 간접적입니다. 교육기관 명칭